# k8s endpoint消失问题

## 参考文献

* [k8s endpoint 消失问题](https://xuxinkun.github.io/2017/09/18/kube-endpoint-disappear/)
* [k8s pod 生命周期](https://kubernetes.io/zh/docs/concepts/workloads/pods/pod-lifecycle/)

## 问题分析

查看service状态

`k get svc streamsets-4 -n bdp`

```
NAME           TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)           AGE
streamsets-4   NodePort   10.99.121.152   <none>        18630:30631/TCP   391d
```

`k describe svc streamsets-4 -n bdp`

```
Name:                     streamsets-4
Namespace:                bdp
Labels:                   run=streamsets-4
Annotations:              kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"labels":{"run":"streamsets-4"},"name":"streamsets-4","namespace":"bdp"},...
Selector:                 run=streamsets-4
Type:                     NodePort
IP:                       10.99.121.152
Port:                     <unset>  18630/TCP
TargetPort:               18630/TCP
NodePort:                 <unset>  30631/TCP
Endpoints:  # 问题点，不应该为空
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>
```

> Note：Endpoints为空

查看endpoint状态

`k get ep streamsets-4 -nbdp`

```
NAME           ENDPOINTS   AGE
streamsets-4               391d
```


` k get ep streamsets-4 -nbdp -oyaml`

```yaml
apiVersion: v1
kind: Endpoints
metadata:
  annotations:
    endpoints.kubernetes.io/last-change-trigger-time: "2022-01-18T02:04:48Z"
  creationTimestamp: "2020-12-23T06:20:18Z"
  labels:
    run: streamsets-4
  name: streamsets-4
  namespace: bdp
  resourceVersion: "154825784"
  selfLink: /api/v1/namespaces/bdp/endpoints/streamsets-4
  uid: 81206da0-f4df-47e5-a00f-b4f3ff87810a
subsets:
- notReadyAddresses:
  - ip: 10.244.89.167
    nodeName: k8s-node-03
    targetRef:
      kind: Pod
      name: streamsets-4-87db655c6-mggt4
      namespace: bdp
      resourceVersion: "154825779"
      uid: 1999349e-43c6-4d79-b4a6-68886c2df848
  ports:
  - port: 18630
    protocol: TCP
```

参考文献https://xuxinkun.github.io/2017/09/18/kube-endpoint-disappear/ 分析是pod状态异常

查看pod、deployment状态

```
[root@k8s-master-01 zhaobingfeng]# k get pod -n bdp -lrun=streamsets-4
NAME                           READY   STATUS    RESTARTS   AGE
streamsets-4-87db655c6-mggt4   1/1     Running   1          57d

[root@k8s-master-01 zhaobingfeng]# k get deployment streamsets-4 -n bdp
NAME           READY   UP-TO-DATE   AVAILABLE   AGE
streamsets-4   0/1     1            0           391d
```

` k get pod -n bdp -lrun=streamsets-4 -oyaml`

```yaml
...

  status:
    conditions:
    - lastProbeTime: null
      lastTransitionTime: "2021-11-22T06:33:17Z"
      status: "True"
      type: Initialized
    - lastProbeTime: null
      lastTransitionTime: "2022-01-18T02:04:48Z"
      status: "False" # 问题点，应该是True
      type: Ready
    - lastProbeTime: null
      lastTransitionTime: "2021-11-22T06:40:59Z"
      status: "True"
      type: ContainersReady
    - lastProbeTime: null
      lastTransitionTime: "2021-11-22T06:33:17Z"
      status: "True"
      type: PodScheduled
    containerStatuses:
    - containerID: docker://89b380c712fc811066808c42e51cd1e702ddebd92c953113247faadfdd8d8d63
      image: 10.100.31.135:5000/com.credithc/streamset-service:v2
      imageID: docker-pullable://10.100.31.135:5000/com.credithc/streamset-service@sha256:5277ea055faa9b14339d4382e7e5f8b07c3e4794fa61cac8774c3d72e99bc4bc
      lastState:
        terminated:
          containerID: docker://f19d56826b10f3c510d092e3444f427ceb5f43528b033564cc57ee2f5c136e83
          exitCode: 137
          finishedAt: "2021-11-22T06:39:56Z"
          reason: Error
          startedAt: "2021-11-22T06:33:24Z"

...
```

> type: Ready 的status为False

## 问题解决

删除pod重启